In [3]:
import mimetypes
from email import policy
from email.parser import BytesParser
import html2text
import imaplib
import datetime, time

In [6]:
with open('test.eml', 'rb') as fp:
    msg = BytesParser(policy=policy.default).parse(fp)

In [7]:
print('To:', msg['to'])
print('From:', msg['from'])
print('Subject:', msg['subject'])

To: Internet Archive <donations@archive.org>
From: Patentanwalt Götz Thomas via PayPal <service@paypal.com>
Subject: Notification of payment received


In [198]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    no_html = re.sub(cleanr, '', raw_html)
    ws_r = re.compile('\s')
    no_ws = re.sub(ws_r, '', no_html)
    return no_html

In [46]:
simplest = msg.get_body(preferencelist=('plain', 'html'))

In [103]:
html2text.config.IGNORE_TABLES = True
html2text.config.IGNORE_IMAGES = False
h = html2text.HTML2Text()
h.ignore_links = True
h.handle(simplest.get_content())[:300]

'#### Internet Archive, here are the details.\n\nHello, Internet Archive  \n![](https://www.paypalobjects.com/digitalassets/c/system-triggered-\nemail/n/layout/images/dark-mode/pplogo-circletop-sm.png)  \n![PayPal](https://www.paypalobjects.com/digitalassets/c/system-triggered-\nemail/n/layout/images/dark-'

In [102]:
cleanhtml(simplest.get_content())[:300]

"\n\n  \n    \n    \n    Notification of payment received\n    \n      /**\n * PayPal Fonts\n */\n      @font-face {\n        font-family: PayPal-Sans;\n        font-style: normal;\n        font-weight: 400;\n        src: local('PayPalSansSmall-Regular'), url('https://www.paypalobjects.com/ui-web/paypal-sans-small"

In [83]:
import imaplib

In [14]:
# MAIL_PASSWORD = input('mail password: ')

In [15]:
server = imaplib.IMAP4_SSL(host='mail.archive.org', port=993)
server.noop()
server.login('support.tickets.bot@archive.org', MAIL_PASSWORD)

('OK', [b'Logged in'])

In [22]:
server.select('FROM_ARCHIVE_ACCOUNTS_0')
status, response = server.search(None, 'ALL')
if status != 'OK':
    print('crap')
    exit(1)
msg_ids = response[0].decode().split(' ')
len(msg_ids)
msg_ids

['1']

In [23]:
status, response = server.fetch(','.join(['2']), '(BODY[])')
if status != 'OK':
    print('crap')
    exit(1)

error: FETCH command error: BAD [b'Error in IMAP command FETCH: Invalid messageset (0.000 + 0.000 secs).']

In [20]:
response

[(b'1 (FLAGS (\\Seen) BODY[] {12190}',
  b'Return-Path: <ella@i-gamingmedia.com>\r\nDelivered-To: support.tickets.bot@archive.org\r\nReceived: from mail.archive.org (localhost [127.0.0.1])\r\n\tby mail.archive.org (Postfix) with ESMTP id 6A931212C5;\r\n\tMon, 14 Dec 2020 17:25:50 +0000 (UTC)\r\nX-Spam-Checker-Version: SpamAssassin 3.4.2 (2018-09-13) on mail.archive.org\r\nX-Spam-Level: *\r\nX-Spam-Status: No, score=2.0 required=5.0 tests=BAYES_50,DKIM_SIGNED,\r\n\tDKIM_VALID,HTML_MESSAGE,RCVD_IN_DNSWL_NONE,RCVD_IN_MSPIKE_H3,\r\n\tRCVD_IN_MSPIKE_WL,SPF_HELO_NONE,SPF_PASS,UNPARSEABLE_RELAY\r\n\tautolearn=no autolearn_force=no version=3.4.2\r\nReceived: from mail-ej1-f65.google.com (mail-ej1-f65.google.com [209.85.218.65])\r\n\t(using TLSv1.2 with cipher ECDHE-RSA-AES128-GCM-SHA256 (128/128 bits))\r\n\t(No client certificate requested)\r\n\tby mail.archive.org (Postfix) with ESMTPS id AD56820D4F\r\n\tfor <info@archive.org>; Mon, 14 Dec 2020 17:25:49 +0000 (UTC)\r\nReceived: by mail-ej1-f6

In [201]:
html2text.config.IGNORE_TABLES = True
html2text.config.IGNORE_IMAGES = False
h = html2text.HTML2Text()
h.ignore_links = True
support_msgs = []
for msg in map(lambda p: BytesParser(policy=policy.default).parsebytes(p[1]), filter(lambda p: len(p) == 2, response)):
    time_str = msg['Received'].split(';')[-1].strip()
    time_str_format = '%a,  %d %b %Y %H:%M:%S %z (%Z)'
    time_object = time.strptime(time_str, time_str_format)
    time_stamp = int(time.mktime(time_object))
    
    simplest = msg.get_body(preferencelist=('plain', 'html'))
    body = h.handle(simplest.get_content())
    
    support_msgs.append((time_stamp, body))

In [202]:
response[:3]

[(b'1 (BODY[] {60049}',
  b'Return-Path: <service@paypal.com>\r\nDelivered-To: support.tickets.bot@archive.org\r\nReceived: from mail.archive.org (localhost [127.0.0.1])\r\n\tby mail.archive.org (Postfix) with ESMTP id 21CB11E96F;\r\n\tTue,  1 Dec 2020 23:20:00 +0000 (UTC)\r\nX-Spam-Checker-Version: SpamAssassin 3.4.2 (2018-09-13) on mail.archive.org\r\nX-Spam-Level: \r\nX-Spam-Status: No, score=-14.2 required=5.0 tests=BAYES_00,DKIM_SIGNED,\r\n\tDKIM_VALID,DKIM_VALID_AU,DKIM_VALID_EF,ENV_AND_HDR_SPF_MATCH,\r\n\tFREEMAIL_FORGED_REPLYTO,FREEMAIL_REPLYTO_END_DIGIT,HTML_IMAGE_RATIO_02,\r\n\tHTML_MESSAGE,MIME_HTML_ONLY,RCVD_IN_MSPIKE_H3,RCVD_IN_MSPIKE_WL,\r\n\tSPF_HELO_NONE,SPF_PASS,T_REMOTE_IMAGE,USER_IN_DEF_DKIM_WL,\r\n\tUSER_IN_DEF_SPF_WL autolearn=no autolearn_force=no version=3.4.2\r\nReceived: from mx2.slc.paypal.com (mx0.slc.paypal.com [173.0.84.225])\r\n\t(using TLSv1.2 with cipher ECDHE-RSA-AES256-GCM-SHA384 (256/256 bits))\r\n\t(No client certificate requested)\r\n\tby mail.archi

In [ ]:
b'63 (BODY[] {59238}'
re.compile('')

In [133]:
status, response = server.fetch(msg_ids[0], '(BODY[])')
if status != 'OK':
    print('crap')
    exit(1)

In [135]:
msg = BytesParser(policy=policy.default).parsebytes(response[0][1])

In [142]:
print(msg['Received'])
print(msg['Date'])
msg.keys()

from mail.archive.org (localhost [127.0.0.1])	by mail.archive.org (Postfix) with ESMTP id C0F6C20ADE	for <support.tickets.bot@archive.org>; Fri, 11 Dec 2020 18:59:39 +0000 (UTC)
Fri, 11 Dec 2020 18:59:39 +0000


['Return-Path',
 'Delivered-To',
 'Received',
 'X-Spam-Checker-Version',
 'X-Spam-Level',
 'X-Spam-Status',
 'Received',
 'Received',
 'Date',
 'From',
 'To',
 'Message-ID',
 'Subject',
 'Mime-Version',
 'Content-Type',
 'Content-Transfer-Encoding',
 'X-Delivery-Context',
 'Auto-Submitted',
 'X-Auto-Response-Suppress',
 'X-Mailer',
 'X-Zendesk-From-Account-Id',
 'DKIM-Signature',
 'X-Envelope-From']

In [123]:
simplest = msg.get_body(preferencelist=('plain', 'html'))

In [124]:
html2text.config.IGNORE_TABLES = True
html2text.config.IGNORE_IMAGES = False
h = html2text.HTML2Text()
h.ignore_links = True
h.handle(simplest.get_content())[:300]

"_**  Internet banking notification**_ ___ _  \n\n\nDans le cadre de nos efforts continus envers vous offrir un excellent service\nclient et d'assurer l'efficacité dans le traitement des transactions, nous\nvous demandons de bien vouloir mettre à jour vos dossiers avec la Banque en\ntéléchargeant le formul"

In [206]:
time.time()

1607724975.150305

In [208]:
t = time.time()

In [ ]:
t.